In [1]:
library(benchmarkme)
get_platform_info()$OS.type
get_r_version()$version.string
get_cpu()$model_name;get_cpu()$no_of_cores
get_ram()

[1] "windows"

[1] "R version 3.5.1 (2018-07-02)"

[1] "Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz"

[1] 8

8.59 GB

In [2]:
##Package was modified to allow complete cross mating
#library("devtools")
#install_github("gglinzijie/xbreed")
library("xbreed")

("|-----------------------------------------------------|")
("|                      xbreed                         |")
("|    Genomic simulation of purebreds and crossbreds   |")
("|               March 2017 Version 1.0.1              |")
("|                                                     |")
("|             H.Esfandyari,A.C.Sorensen               |")
("| Center for Quantitative Qenetics and Genomics (QGG) |")
("|             Aarhus University,Denmark               |")
("|                                                     |")
("|-----------------------------------------------------|")
("|Questions and bugs: esfandyari.hadi@gmail.com        |")
("|Development of xbreed was supported by GenSAP.       |")
("|-----------------------------------------------------|")


# Genome specification

In [6]:
#Number of markers per chr from chapter 1
lin_map<-read.table("raw.map")
m=1:22
for(i in 1:22) {m[i]=dim(lin_map[lin_map$V1==i,])[1]}
sum(m)

[1] 3928

In [4]:
#data from chaptre 1
geno<-read.table("Hetero_realigned_cov10_filtered3.raw")
pheno<-read.csv("2017heteroPheno.csv", header=T)
attach(pheno)

In [5]:
#parameter of genome
no.chr<-22
genome<-data.frame(matrix(NA, nrow=no.chr, ncol=6))
names(genome)<-c("chr","len","nmrk","mpos","nqtl","qpos")
genome$chr<-c(1:no.chr) #Chromosome id from 1 to 22
genome$len<-c(200,rep(100,21))#Chromosome length in cM
genome$nmrk<-c(3*m) #Number of markers, 3*3928 in total 
genome$mpos<-c('even') 
genome$nqtl<-c(40) #Number of qtl  40*22 = 880 in total
genome$qpos<-c('rnd')

# Historiacal population

In [6]:
#Historical population 
hp<-make_hp(hpsize=1000 ,ng=5000,h2=0.654,d2=0,phen_var=84,
            genome=genome,mutr=2.5*10**-4,laf=1)

---sel_seq_qtl is missing, it has been set to default value of 0
---sel_seq_mrk is missing, it has been set to default value of 0
Historical pop is initialized...
Simulating trait ...
Output data preparation ...
Establishment of historical population completed


In [7]:
#validation
mutr<-2.5*10**-4
ne<-1000
k<-2 
Fneu<-4*ne*mutr
(Expected_het1<-1-((1+((Fneu)/(k-1)))/(1+((Fneu*k)/(k-1)))))
(het_observed<-mean(2*(hp$freqMrk[,3]*hp$freqMrk[,4])))

[1] 0.3333333

[1] 0.398942

In [8]:
Male_founders<-data.frame(number=10,select='rnd')
Female_founders<-data.frame(number=10,select='rnd')

In [9]:
Selection<-data.frame(matrix(NA, nrow=2, ncol=3))
names(Selection)<-c('Number','type','Value') 
Selection$Number[1:2]<-c(10,10)
Selection$type[1:2]<-c('rnd','rnd')
Selection$Value[1:2]<-c('l','l')

In [10]:
sh_output<-data.frame(matrix(NA, nrow=1, ncol=4))
names(sh_output)<-c("data","qtl","freq_mrk","marker")
sh_output[1]<-c(1) 
sh_output[2]<-c(1) 
sh_output[3]<-c(1)
sh_output[4]<-c(1)

In [11]:
#10 sires and 10 dams perform complete cross; 200 eggs were produced per dam.
RP<-sample_hp(hp_out=hp,Male_founders= Male_founders,
              Female_founders=Female_founders,ng=1,Selection=Selection, 
              litter_size=10,saveAt="SNP3928",sh_output=sh_output,Display=FALSE)

Controlling input data ...
Intializing base population ...
Generation 0 started ......... 
Generation 0 is finished. Time taken: 14.86296
Generation 1 started ......... 
Generation 1 is finished. Time taken: 21.10412
Output data preparation ...
Writing output files ...
Sampling hp is done!


In [12]:
#function for calculate the allele coding, which is 0, 1, 2
bin_snp<-function(mat){
s1<-seq(1,ncol(mat),2)
s2<-seq(2,ncol(mat),2)
a1<-mat[,s1]+mat[,s2]
a1[a1==3]=1
a1[a1==4]=0
snp_code<-a1
return(snp_code)
 }

In [13]:
##geno and pheno from simulated data
pheno<-RP$output[[2]]$data$phen
tbvp<-RP$output[[2]]$data$tbvp
n<-bin_snp(RP$output[[2]]$mrk[,3:23570])
x<-as.matrix(n)-1
colnames(x)<-1:(3928*3)
row.names(x)<-1:1000

In [14]:
write.table(x,"2.4_geno.txt")
write.table(pheno,"2.4_pheno.txt")
write.table(tbvp,"2.4_tbvp.txt")

In [1]:
# Simulation result
x<-as.matrix(read.table("2.4_geno.txt"))
pheno<-as.numeric(unlist(read.table("2.4_pheno.txt")))
tbvp<-as.numeric(unlist(read.table("2.4_tbvp.txt")))

In [2]:
##load packages
library(doParallel)
library(foreach)
cl<-makeCluster(8) 
repeats <- 10
n.fold <- 5 
acc<-list()
library(rrBLUP)
packageVersion("rrBLUP") 

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


[1] '4.6'

In [3]:
cal_se<-function(acc){sd(acc)/sqrt(repeats-1)}

In [4]:
# Function for calculating the accuracy of GP with 
# varying size of reference population and varying denstiy of SNP panel
cal_acc<-function(x,n.sample,n.snp){
set.seed(100)
id<-sample(1:dim(x)[1],n.sample)
snpId<-sample(1:dim(x)[2],n.snp)
#relationship matrix (Endelman at al. 2011)
A <- A.mat(x[id,snpId], n.core=8)
row.names(A)=1:n.sample;colnames(A)=1:n.sample
data <- data.frame(tbcw=pheno[id],tbvp=tbvp[id],gid=1:n.sample)
result<-data.frame(heritability=rep(NA,50),
                   bias_r=rep(NA,50),
                   unbias_r=rep(NA,50),
                   true_r=rep(NA,50))    
registerDoParallel(cl)
foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="rrBLUP") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         res <- kin.blup(bcw_test, K=A, geno="gid", pheno="tbcw")
         result[5*(j-1)+i,1]<-res$Vg/(res$Vg+res$Ve)
         result[5*(j-1)+i,2]<-cor(data$tbcw[id==i],res$pred[id==i])
         result[5*(j-1)+i,3]<-cor(data$tbcw[id==i],res$pred[id==i])/sqrt(res$Vg/(res$Vg+res$Ve))
         result[5*(j-1)+i,4]<-cor(data$tbvp[id==i],res$pred[id==i])
    }
   }
stopImplicitCluster()
return(c(apply(result, 2, mean),apply(result,2,cal_se)))    
}

In [21]:
sum_m<-sum(m)
n.snp<-c(round(0.1*sum_m),round(0.2*sum_m), 
             round(0.3*sum_m),round(0.4*sum_m),
             round(0.5*sum_m),round(0.6*sum_m),
             round(0.7*sum_m),round(0.8*sum_m),
             round(0.9*sum_m),sum_m,
             2*sum_m, 3*sum_m)
             
n.sample<-rep(c(1000),12)
pair<-data.frame(n.snp=n.snp,n.sample=n.sample)

In [22]:
#implemente the caculation 
all_acc<-matrix(NA,nrow = 8,ncol = length(n.snp))
all_accBA<-all_accBB<-all_accBC<-all_accBL<-all_acc
for (i in 1:length(n.snp)){
    all_acc[,i]<-cal_acc(x,n.sample[i],n.snp[i])}

In [23]:
colnames(all_acc)<-n.snp
row.names(all_acc)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")

In [24]:
library("BGLR")
packageVersion("BGLR") 

[1] '1.0.5'

In [25]:
# Function for calculating the accuracy of GP with 
# varying size of reference population and varying denstiy of SNP panel
cal_accB<-function(x,n.sample,n.snp,M){
set.seed(100)
id<-sample(1:dim(x)[1],n.sample)
snpId<-sample(1:dim(x)[2],n.snp)
x1<-x[id,snpId]
data <- data.frame(tbcw=pheno[id],tbvp=tbvp[id],gid=1:n.sample)
result<-data.frame(heritability=rep(NA,50),
                   bias_r=rep(NA,50),
                   unbias_r=rep(NA,50),
                   true_r=rep(NA,50))                   
registerDoParallel(cl)
foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fm=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x1,model=M,saveEffects=T,saveAt='fm_')),nIter=2000,burnIn=1000,verbose=F)
         varU=var(x1%*%fm$ETA[[1]]$b)
         varE=fm$varE
         h2=varU/(varU+varE)
         result[5*(j-1)+i,1]<-h2
         result[5*(j-1)+i,2]<-cor(data$tbcw[id == i],fm$yHat[id == i])
         result[5*(j-1)+i,3]<-cor(data$tbcw[id==i],fm$yHat[id == i])/sqrt(h2)
         result[5*(j-1)+i,4]<-cor(data$tbvp[id==i],fm$yHat[id == i])
    }
   }
stopImplicitCluster()
return(c(apply(result, 2, mean),apply(result,2,cal_se)))    
}

In [26]:
for (i in 1:length(n.snp)){all_accBA[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BayesA")}

In [27]:
for (i in 1:length(n.snp)){all_accBB[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BayesB")}

In [28]:
for (i in 1:length(n.snp)){all_accBC[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BayesC")}

In [29]:
for (i in 1:length(n.snp)){all_accBL[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BL")}

Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1582 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1583 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1584 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1585 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1586 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 15

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1630 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1631 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1632 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1633 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1634 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1678 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1679 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1680 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1681 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1682 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1726 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1727 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1728 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1729 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1730 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1774 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1775 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1776 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1777 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1778 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

...

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1794 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1795 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1796 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1797 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1798 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1842 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1843 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1844 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1845 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1846 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1890 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1891 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1892 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1893 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1894 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1938 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1939 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1940 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1941 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1942 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1986 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1987 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1988 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1989 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1990 due to numeric problems with beta
"Warning message in BGLR(y = bcw_test$tbcw, ETA = list(list(X = x1, model = M, saveEffects = T, :
"tau2 was not updated  in iteration 1

In [30]:
colnames(all_accBA)<-n.snp
colnames(all_accBB)<-n.snp
colnames(all_accBC)<-n.snp
colnames(all_accBL)<-n.snp
row.names(all_accBA)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBB)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBC)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBL)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")

In [31]:
(sum<-list(GBLUP=all_acc,
                BA=all_accBA,
                BB=all_accBB,
                BC=all_accBC,
                BL=all_accBL))

,393,786,1178,1571,1964,2357,2750,3142,3535,3928,7856,11784
h2,0.332216021,0.40293565,0.439626571,0.456032054,0.485166238,0.501377575,0.513899512,0.51770405,0.528467210,0.53765573,0.59428767,0.60629281
bias_r,0.468197720,0.50498791,0.521177485,0.527916718,0.534592675,0.539575848,0.541988329,0.54257706,0.542656247,0.54368519,0.55528773,0.55612309
unbias_r,0.815798124,0.79818220,0.788209761,0.783748338,0.769329056,0.763916843,0.757984387,0.75612936,0.748516868,0.74348830,0.72204475,0.71574182
true_r,0.612458504,0.66226163,0.677923841,0.687313148,0.697336951,0.704936339,0.708510603,0.71217801,0.713544030,0.71489413,0.72667594,0.72674749
h2_se,0.007430797,0.00791970,0.008008934,0.007958384,0.008306891,0.008539445,0.009225537,0.00953815,0.009957006,0.01025381,0.01111181,0.01073074
bias_r_se,0.018153021,0.01768139,0.016959220,0.017266246,0.017090859,0.017188979,0.017061143,0.01715783,0.017217408,0.01710149,0.01604665,0.01589257
unbias_r_se,0.038315203,0.03354002,0.030407474,0.030024339,0.028736315,0.028703041,0.028296968,0.02855947,0.028314967,0.02790154,0.02491483,0.02402548
true_r_se,0.015971056,0.01627303,0.015454514,0.014542902,0.013800710,0.013787583,0.013730146,0.01383534,0.014051869,0.01433374,0.01262129,0.01243065
,393,786,1178,1571,1964,2357,2750,3142,3535,3928,7856,11784
h2,0.262102327,0.322053259,0.357396354,0.364252904,0.39076696,0.393584527,0.40627401,0.40952535,0.41866413,0.42016119,0.47314305,0.48627122


In [32]:
#save to xlsx file
library("xlsx")
write.xlsx(sum, "2_4_result1000.xlsx") 

In [ ]:
library("readxl")
plot<-read_excel("2_4_result.xlsx")